In [ ]:
# make_viz_fast_map.py
# Plotly + Map tiles (OpenStreetMap) with Time/History sliders + Play/Pause.
# Performance-optimized: time-bucketed segments (fast updates), light bg path downsampling.

import json, math, pandas as pd
import plotly.graph_objects as go
from plotly.io import to_html

# --------- Settings ----------
DATA_FILE = r"C:\Hackathon\crowd_demo_escape_lastest_preserve15.json"
OUTPUT_HTML = r"C:\Hackathon\tracks_escape_latest_speed.html"
SPEED_THRESH_MPS = 2.7     # ~6 mph
TITLE = "Crowd Motion — red = speed > 2.7 m/s"
SEG_WIDTH = 3
BG_PATH_COLOR = "lightgray"
SLOW_COLOR = "royalblue"
FAST_COLOR = "red"
DEFAULT_HISTORY_S = 10.0    # initial preserved history
TIME_STEP = 10              # slider & animation step (seconds)
TICK_MS = 250               # play timer interval (ms)
MAP_STYLE = "open-street-map"   # try "carto-positron" too
BG_MAX_POINTS_PER_PHONE = 400   # cap per-phone background path density
COORD_ROUND = 6    
# -----------------------------

# 1) Load data
with open(DATA_FILE, "r", encoding="utf-8") as f:
    records = json.load(f)

rows = []
for obj in records:
    pid = obj["phone_id"]
    for p in obj["track"]:
        rows.append({
            "phone_id": pid,
            "t": p["t"],
            "lon": p["lon"],
            "lat": p["lat"],
            "speed": p.get("speed", None)
        })
df = pd.DataFrame(rows)
df["t"] = pd.to_numeric(df["t"], errors="coerce")
df = df.dropna(subset=["lon","lat"]).copy()
df = df.sort_values(["phone_id","t"]).reset_index(drop=True)

# 2) Build segments list once (and compute time bounds)
segs = []
bg_paths = {}
for pid, g in df.groupby("phone_id"):
    g = g.sort_values("t")
    xs = g["lon"].round(COORD_ROUND).to_list()
    ys = g["lat"].round(COORD_ROUND).to_list()

    # Downsample background path to limit DOM size
    if len(xs) > BG_MAX_POINTS_PER_PHONE:
        stride = max(1, len(xs) // BG_MAX_POINTS_PER_PHONE)
        xs = xs[::stride]
        ys = ys[::stride]
    bg_paths[pid] = (xs, ys)

    for i in range(1, len(g)):
        x0, y0, t0 = g.iloc[i-1][["lon","lat","t"]]
        x1, y1, t1 = g.iloc[i][["lon","lat","t"]]
        spd = g.iloc[i]["speed"]
        over = (pd.notna(spd) and spd > SPEED_THRESH_MPS)
        segs.append({
            "pid": int(pid) if str(pid).isdigit() else pid,
            "x0": float(round(x0, COORD_ROUND)), "y0": float(round(y0, COORD_ROUND)),
            "x1": float(round(x1, COORD_ROUND)), "y1": float(round(y1, COORD_ROUND)),
            "t_end": float(t1),
            "speed": None if pd.isna(spd) else float(spd),
            "over": bool(over)
        })

t_min = min((s["t_end"] for s in segs), default=0.0)
t_max = max((s["t_end"] for s in segs), default=0.0)

# Add frame index now (reduces JS work)
for s in segs:
    # Frame numbers start at 0
    s["f"] = int(max(0, math.floor((s["t_end"] - t_min) / TIME_STEP)))

# 3) Base figure on map tiles
fig = go.Figure()

# Background paths (one per phone; light & cheap)
for pid, (xs, ys) in bg_paths.items():
    fig.add_trace(go.Scattermapbox(
        lon=xs, lat=ys, mode="lines",
        line=dict(width=1, color=BG_PATH_COLOR),
        hoverinfo="skip", showlegend=False, name=f"path_{pid}"
    ))
bg_count = len(bg_paths)

# Two *empty* segment traces: slow (blue) and fast (red)
fig.add_trace(go.Scattermapbox(
    lon=[], lat=[], mode="lines",
    line=dict(width=SEG_WIDTH, color=SLOW_COLOR),
    hoverinfo="skip", showlegend=False, name="slow_segments"
))
fig.add_trace(go.Scattermapbox(
    lon=[], lat=[], mode="lines",
    line=dict(width=SEG_WIDTH, color=FAST_COLOR),
    hoverinfo="skip", showlegend=False, name="fast_segments"
))
idx_slow = bg_count + 0
idx_fast = bg_count + 1

# Map center/zoom heuristic
min_x, max_x = df["lon"].min(), df["lon"].max()
min_y, max_y = df["lat"].min(), df["lat"].max()
pad_x = (max_x - min_x) * 0.05 if max_x > min_x else 0.01
pad_y = (max_y - min_y) * 0.05 if max_y > min_y else 0.01
center_lon = float((min_x + max_x) / 2.0)
center_lat = float((min_y + max_y) / 2.0)
span = max((max_x - min_x) + 2*pad_x, (max_y - min_y) + 2*pad_y, 0.001)
zoom = max(1, min(18, math.log2(360.0 / span) - 0.5))

fig.update_layout(
    title=TITLE,
    mapbox=dict(
        style=MAP_STYLE,
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom
    ),
    margin=dict(l=10, r=10, t=60, b=10),
)

# 4) Embed figure + lightweight controls & fast JS updater
fig_html = to_html(fig, include_plotlyjs="cdn", full_html=False, div_id="viz")

max_trail = max(1.0, t_max - t_min)

controls = f"""
<div style="margin:10px 0; font-family:system-ui, -apple-system, Segoe UI, Roboto, sans-serif;">
  <label>Time (s):
    <input type="range" id="timeSlider" min="{t_min:.2f}" max="{t_max:.2f}" step="{TIME_STEP}" value="{t_min:.2f}" style="width:320px;">
    <span id="timeVal">{t_min:.2f}</span>
  </label>
  <label style="margin-left:24px;">History (s):
    <input type="range" id="trailSlider" min="0" max="{max_trail:.2f}" step="{TIME_STEP}" value="{DEFAULT_HISTORY_S:.2f}" style="width:240px;">
    <span id="trailVal">{DEFAULT_HISTORY_S:.2f}</span>
  </label>
  <button id="playBtn" style="margin-left:24px;">▶ Play</button>
  <button id="pauseBtn">⏸ Pause</button>
</div>
"""

import json as pyjson
segs_js = pyjson.dumps(segs, separators=(",", ":"))  # compact

script = f"""
<script>
(function(){{
  const gd = document.getElementById("viz");
  const timeSlider = document.getElementById("timeSlider");
  const trailSlider = document.getElementById("trailSlider");
  const timeVal = document.getElementById("timeVal");
  const trailVal = document.getElementById("trailVal");
  const playBtn = document.getElementById("playBtn");
  const pauseBtn = document.getElementById("pauseBtn");

  const segs = {segs_js};  // precomputed segments with frame index 'f'
  const idxSlow = {idx_slow};
  const idxFast = {idx_fast};
  const tickMs = {TICK_MS};
  const stepSec = {TIME_STEP};
  const tMin = parseFloat(timeSlider.min);

  let timer = null;

  // ---- One-time bucketing by frame ----
  // We'll store lon/lat arrays per frame for slow/fast.
  const slowByF_lon = [];
  const slowByF_lat = [];
  const fastByF_lon = [];
  const fastByF_lat = [];

  let fMax = 0;
  for (let i=0; i<segs.length; i++) {{
    const s = segs[i];
    const f = s.f;
    if (!slowByF_lon[f]) {{
      slowByF_lon[f] = []; slowByF_lat[f] = [];
      fastByF_lon[f] = []; fastByF_lat[f] = [];
    }}
    if (s.over) {{
      fastByF_lon[f].push(s.x0, s.x1, null);
      fastByF_lat[f].push(s.y0, s.y1, null);
    }} else {{
      slowByF_lon[f].push(s.x0, s.x1, null);
      slowByF_lat[f].push(s.y0, s.y1, null);
    }}
    if (f > fMax) fMax = f;
  }}

  function frameFromTime(t) {{
    const dt = t - tMin;
    return Math.max(0, Math.floor(dt / stepSec));
  }}

  function concatWindow(fStart, fEnd) {{
    const sLon = [], sLat = [], fLon = [], fLat = [];
    for (let f = fStart; f <= fEnd; f++) {{
      if (slowByF_lon[f]) {{
        sLon.push.apply(sLon, slowByF_lon[f]);
        sLat.push.apply(sLat, slowByF_lat[f]);
      }}
      if (fastByF_lon[f]) {{
        fLon.push.apply(fLon, fastByF_lon[f]);
        fLat.push.apply(fLat, fastByF_lat[f]);
      }}
    }}
    return {{ sLon, sLat, fLon, fLat }};
  }}

  function updateUI() {{
    const t = parseFloat(timeSlider.value);
    const trail = parseFloat(trailSlider.value);
    timeVal.textContent = t.toFixed(2);
    trailVal.textContent = trail.toFixed(2);

    // Convert to frame window
    const fEnd = frameFromTime(t);
    const trailFrames = Math.max(0, Math.floor(trail / stepSec));
    const fStart = Math.max(0, fEnd - trailFrames);

    const {{ sLon, sLat, fLon, fLat }} = concatWindow(fStart, fEnd);
    Plotly.restyle(gd, {{lon: [sLon], lat: [sLat]}}, [idxSlow]);
    Plotly.restyle(gd, {{lon: [fLon], lat: [fLat]}}, [idxFast]);
  }}

  timeSlider.addEventListener('input', updateUI);
  trailSlider.addEventListener('input', updateUI);

  playBtn.addEventListener('click', function() {{
    if (timer) return;
    timer = setInterval(function() {{
      let t = parseFloat(timeSlider.value) + stepSec;
      if (t > parseFloat(timeSlider.max)) t = parseFloat(timeSlider.min);
      timeSlider.value = t.toFixed(2);
      updateUI();
    }}, tickMs);
  }});

  pauseBtn.addEventListener('click', function() {{
    if (timer) {{ clearInterval(timer); timer = null; }}
  }});

  // Initial draw
  updateUI();
}})();
</script>
"""

html = f"""<!doctype html>
<html>
<head><meta charset="utf-8"><title>Crowd Demo (Map)</title></head>
<body>
{controls}
{fig_html}
{script}
</body>
</html>
"""

with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print(f"Saved {OUTPUT_HTML}")


Saved C:\Hackathon\tracks_escape_latest_speed.html
